In [184]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
# from sklearn.cluster import KMeans
from scipy.cluster.vq import *
import random
from sklearn.svm import LinearSVC

In [185]:
def objectDetection(videofile):
    cap = cv2.VideoCapture(videofile)
    print("The video has  {0} frames".format(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
#     print("The video is playing at {} frames per second.".format(cap.get(cv2.CV_CAP_PROP_FPS)))
    # Trained XML classifiers describes some features of some object we want to detect
    car_cascade = cv2.CascadeClassifier('cars.xml')

    while(True):
        # Capture frame-by-frame
        ret, frame = cap.read()

        if ret == False:
            break

        # Our operations on the frame come here
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detects cars of different sizes in the input image
        cars = car_cascade.detectMultiScale(gray, 1.1, 1)

        # To draw a rectangle in each cars
        for (x,y,w,h) in cars:
            cv2.rectangle(gray,(x,y),(x+w,y+h),(0,0,255),2)

        # Display the resulting frame
        cv2.imshow('frame',gray)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()

In [205]:
def tracking(videofile):
    cap = cv2.VideoCapture(videofile)

    ret, frame1 = cap.read()
    prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    hsv = np.zeros_like(frame1)
    hsv[...,1] = 255
    features = []

    while(1):
        ret, frame2 = cap.read()

        if(ret == False):
            break

        next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
        print(np.array(next).shape)

        flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
        hsv[...,0] = ang*180/np.pi/2
        hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
        rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
        
#         mser = cv2.MSER_create()
#         regions = mser.detectRegions(mag, None)
        
#         print(len(regions))
        
        features.append(np.partition(-mag, 50).flatten())
#         features.append(mag[mag > 0.05])

        cv2.imshow('frame2',rgb)
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
       
        prvs = next   
    
#     print(features)
#     print(len(features))
    features = random.sample(features, 200); 
    cap.release()
    cv2.destroyAllWindows()
    return features

In [206]:
def generateCodebook(features, k):
    X = np.array(features[0][1])
    for path, feature in features[1:]:
        X = np.vstack((X, feature))
        
    print(len(X))
    codebook, variance = kmeans(X, k, 1) 
    return codebook

In [207]:
def generateBOW(features, codebook):
    bow = np.zeros((len(features), len(codebook)), "float32")
    i = 0
    for path, feature in features[0:]:
        words, d = vq(feature,codebook)
        for w in words:
            bow[i][w] += 1
        i = i + 1
    
    return bow

In [208]:
def extractFeatures():
    allfeatures = [];
    f = range(1,4)
    k = 25
    
    for fn in f:
        videofile = 'normal/{}.mov'.format(fn)
    #     objectDetection(videofile)
        feature = tracking(videofile)
        allfeatures.append((videofile,feature))
        
    codebook = generateCodebook(allfeatures, k)
    bagOfWordsFeatures = generateBOW(allfeatures, codebook)
    
#     print(bagOfWordsFeatures)
    return bagOfWordsFeatures
    

In [209]:
def trainVideo(features, classes):
    clf = LinearSVC()
    clf.fit(features, classes)
    return clf

In [210]:
def testVideo(model):
    return ""
    

In [211]:
def main():
        features = extractFeatures()
#         split features into train and test
        model = trainVideo(features, [1,1,-1])
        print(len(features))
        print(model.predict(features[1:]))
        testVideo(model)

In [212]:
main()

(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)
(240, 320)

In [139]:
a = range(1, 4) 
for t in a: 
    print(t) 

1
2
3


3